In [348]:
import pandas as pd

In [349]:
food_fndds = pd.read_parquet('fndds_parquet/food.parquet')
food_nutrient_fndds = pd.read_parquet('fndds_parquet/food_nutrient.parquet')
nutrient_fndds = pd.read_parquet('fndds_parquet/nutrient.parquet').set_index('nutrient_nbr')
nutrient_fndds = nutrient_fndds.loc[nutrient_fndds.index.notna()]
# nutrient_fndds.drop(columns=['id', 'rank']).to_parquet('fndds_parquet/nutrient.parquet')
# food_fndds.to_parquet('fndds_parquet/food.parquet')
# food_nutrient_fndds.to_parquet('fndds_parquet/food_nutrient.parquet')
# nutrient_fndds.to_parquet('fndds_parquet/nutrient.parquet')

In [350]:
food_legacy = pd.read_parquet('legacy_parquet/food.parquet')
food_nutrient_legacy = pd.read_parquet('legacy_parquet/food_nutrient.parquet')
nutrient_legacy = pd.read_parquet('legacy_parquet/nutrient.parquet')
id_to_nbd = pd.read_parquet('legacy_parquet/id_to_nbd.parquet')
f_ndb = food_legacy.join(id_to_nbd, how='left').reset_index().set_index('NDB_number')
# food_legacy.to_parquet('legacy_parquet/food.parquet')
# food_nutrient_legacy.to_parquet('legacy_parquet/food_nutrient.parquet')
# nutrient_legacy.to_parquet('legacy_parquet/nutrient.parquet')
# id_to_nbd.to_parquet('legacy_parquet/id_to_nbd.parquet')

In [351]:
codes = pd.read_csv('food_codes.csv', encoding='latin1')
codes = codes.loc[codes.price.notna()]
codes_legacy = codes.loc[codes.fndds_code.isna()].copy()
codes_fndds = codes.loc[codes.fndds_code.notna()].copy()
codes_fndds['fndds_code'] = codes_fndds.fndds_code.astype(int)
codes_legacy = codes_legacy.merge(f_ndb, left_on='ndb_code', right_index=True)
codes_legacy = codes_legacy[['food', 'name', 'price', 'lbs', 'multiplier', 'fdc_id', 'description']]
codes_fndds = codes_fndds.merge(food_fndds.reset_index(), left_on='fndds_code', right_on='food_code')
codes_fndds = codes_fndds[['food', 'name', 'price', 'lbs', 'multiplier', 'fdc_id', 'description']]
codes_fndds = codes_fndds.rename(columns={'food_code': 'fdc_id'})
codes = pd.concat([codes_legacy, codes_fndds]).set_index('fdc_id')

In [352]:
food_nutrient = pd.concat([food_nutrient_fndds, food_nutrient_legacy])
core_nutrients = codes.join(food_nutrient, how='left')

In [353]:
core_nutrients.columns

Index(['food', 'name', 'price', 'lbs', 'multiplier', 'description', 'id',
       'nutrient_id', 'amount', 'data_points', 'derivation_id', 'min', 'max',
       'median', 'footnote', 'min_year_acquired'],
      dtype='object')

In [354]:
core_nutrients['nutrient_id'] = core_nutrients.nutrient_id.astype(float)
core_nutrients = core_nutrients.drop(columns='name')

In [355]:
cnap = core_nutrients.merge(nutrient_fndds.reset_index(), left_on='nutrient_id', right_on='nutrient_nbr', how='inner')
cna = cnap.pivot(index='food', columns=['name','nutrient_nbr'], values='amount')
cna.to_parquet('core_nutrient_amounts_v1.parquet')
cna.droplevel(1, axis=1).sort_values(by='Energy')

name,"Iron, Fe",Water,Vitamin D (D2 + D3),"Sugars, Total",SFA 10:0,SFA 14:0,Energy,Retinol,"Alcohol, ethyl",Vitamin E (alpha-tocopherol),...,SFA 18:0,"Selenium, Se",PUFA 20:5 n-3 (EPA),Folic acid,"Potassium, K",Thiamin,"Folate, DFE","Folate, food","Copper, Cu","Phosphorus, P"
food,,,,,,,,,,,,,,,,,,,,,
Bok Choy,0.80,95.32,0.0,1.18,0.000,0.001,13.0,0.0,0.0,0.09,...,0.001,0.5,0.0,0.0,252.0,0.040,66.0,66.0,0.021,37.0
Lettuce,0.64,95.31,0.0,1.38,0.000,0.000,14.0,0.0,0.0,0.20,...,0.002,0.4,0.0,0.0,168.0,0.056,34.0,34.0,0.027,24.0
Cucumber,0.28,95.23,0.0,1.67,0.000,0.005,15.0,0.0,0.0,0.03,...,0.005,0.3,0.0,0.0,147.0,0.027,7.0,7.0,0.041,24.0
Zucchini,0.37,94.79,0.0,2.50,0.016,0.001,17.0,0.0,0.0,0.12,...,0.006,0.2,0.0,0.0,261.0,0.045,24.0,24.0,0.053,38.0
Swiss Chard,1.80,92.66,0.0,1.10,0.000,0.000,19.0,0.0,0.0,1.89,...,0.000,0.9,0.0,0.0,379.0,0.040,14.0,14.0,0.179,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Almonds,3.73,2.41,0.0,4.86,0.000,0.019,598.0,0.0,0.0,23.90,...,0.704,2.0,0.0,0.0,713.0,0.077,55.0,55.0,1.099,471.0
Hazelnuts,4.70,5.31,0.0,4.34,0.000,0.000,628.0,0.0,0.0,15.03,...,1.265,2.4,0.0,0.0,680.0,0.643,113.0,113.0,1.725,290.0
Walnuts,2.91,4.07,0.0,2.61,0.000,0.000,654.0,0.0,0.0,0.70,...,1.659,4.9,0.0,0.0,441.0,0.341,98.0,98.0,1.586,346.0


In [356]:
cnap['price_per_lb'] = cnap.price / cnap.lbs
cnap['multiplier'] = cnap.multiplier.fillna(1)
cnap['price_per_lb_adjusted'] = cnap.price / (cnap.lbs * cnap.multiplier)
cnap = cnap.loc[:,['food', 'price', 'lbs', 'multiplier', 'description', 'amount',
             'nutrient_nbr', 'name',
       'unit_name', 'price_per_lb', 'price_per_lb_adjusted']]
cnap.to_parquet('core_nutrient_amounts_prices_v1.parquet')

In [362]:
cnap.name.unique()

array(['Iron, Fe', 'Water', 'Vitamin D (D2 + D3)', 'Sugars, Total',
       'SFA 10:0', 'SFA 14:0', 'Energy', 'Retinol', 'Alcohol, ethyl',
       'Vitamin E (alpha-tocopherol)', 'Carotene, alpha',
       'Total lipid (fat)', 'Magnesium, Mg', 'Niacin',
       'Lutein + zeaxanthin', 'PUFA 22:6 n-3 (DHA)',
       'Carbohydrate, by difference', 'Cholesterol',
       'Fatty acids, total polyunsaturated',
       'Fatty acids, total monounsaturated', 'SFA 16:0',
       'Vitamin C, total ascorbic acid', 'SFA 4:0', 'Zinc, Zn',
       'Folate, total', 'Choline, total', 'MUFA 18:1', 'Vitamin E, added',
       'PUFA 18:3', 'Vitamin B-6', 'MUFA 20:1',
       'Vitamin K (phylloquinone)', 'Protein', 'PUFA 18:4', 'PUFA 20:4',
       'SFA 12:0', 'Riboflavin', 'Theobromine', 'Carotene, beta',
       'Lycopene', 'Cryptoxanthin, beta', 'Fiber, total dietary',
       'Vitamin B-12, added', 'SFA 6:0', 'Fatty acids, total saturated',
       'PUFA 18:2', 'Vitamin A, RAE', 'PUFA 22:5 n-3 (DPA)', 'Caffeine',
   

In [363]:
def ppnu(df, nutrient):
    return df.loc[df.name == nutrient].sort_values(by='price_per_nutrient_unit')

In [372]:
cnap.unit_name.unique()
LB_CONVERSIONS = {
    'G': 0.00220462,
    'MG': 0.00000220462,
    'UG': 0.00000000220462,
}

array(['MG', 'G', 'UG', 'KCAL'], dtype=object)

In [374]:
cnap['g'] = cnap.lbs * 453.592
cnap['price_per_g'] = cnap.price / cnap.g
cnap['price_per_g_adjusted'] = cnap.price / (cnap.g * cnap.multiplier)
cnap['price_per_100_g'] = cnap.price_per_g * 100
cnap['price_per_100_g_adjusted'] = cnap.price_per_g_adjusted * 100
cnap['price_per_nutrient_unit'] = cnap.price_per_100_g_adjusted / cnap.amount

In [390]:
ppnu(cnap, 'PUFA 20:5 n-3 (EPA)')

,food,price,lbs,multiplier,description,amount,nutrient_nbr,name,unit_name,price_per_lb,price_per_lb_adjusted,price_per_nutrient_unit,g,price_per_g,price_per_g_adjusted,price_per_100_g,price_per_100_g_adjusted
4858,Salmon,8.00,1.0,0.75,"Fish, salmon, NFS",0.860,629.0,PUFA 20:5 n-3 (EPA),G,8.0000,10.666667,2.734418,453.592,0.017637,0.023516,1.763700,2.351599
4859,Pink Salmon,3.60,1.0,1.00,"Fish, salmon, canned",0.274,629.0,PUFA 20:5 n-3 (EPA),G,3.6000,3.600000,2.896587,453.592,0.007937,0.007937,0.793665,0.793665
4860,Mussels,10.00,1.0,1.00,Mussels,0.238,629.0,PUFA 20:5 n-3 (EPA),G,10.0000,10.000000,9.263128,453.592,0.022046,0.022046,2.204624,2.204624
4861,Oysters,10.00,1.0,1.00,"Oysters, raw",0.177,629.0,PUFA 20:5 n-3 (EPA),G,10.0000,10.000000,12.455505,453.592,0.022046,0.022046,2.204624,2.204624
4862,Shrimp,7.50,1.0,0.75,"Shrimp, NFS",0.074,629.0,PUFA 20:5 n-3 (EPA),G,7.5000,10.000000,29.792222,453.592,0.016535,0.022046,1.653468,2.204624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4875,Peanuts,2.45,1.0,1.00,"Peanuts, NFS",0.000,629.0,PUFA 20:5 n-3 (EPA),G,2.4500,2.450000,inf,453.592,0.005401,0.005401,0.540133,0.540133
4874,Macadamia nuts,15.00,1.0,1.00,Macadamia nuts,0.000,629.0,PUFA 20:5 n-3 (EPA),G,15.0000,15.000000,inf,453.592,0.033069,0.033069,3.306937,3.306937
4873,Hazelnuts,6.80,1.0,1.00,Hazelnuts,0.000,629.0,PUFA 20:5 n-3 (EPA),G,6.8000,6.800000,inf,453.592,0.014991,0.014991,1.499145,1.499145
4880,Quinoa,55.49,25.0,3.00,"Quinoa, no added fat",0.000,629.0,PUFA 20:5 n-3 (EPA),G,2.2196,0.739867,inf,11339.800,0.004893,0.001631,0.489338,0.163113


In [384]:
cnap.to_parquet('core_nutrient_amounts_prices_v2.parquet')